## Record Matching Algorithms

* We will display one possible approach to match records using the existing rltk algorithm and Gale Shapley matching algorithm in this file (unfinished yet)

In [7]:
import rltk
import pandas as pd

In [1]:

ds1 = pd.read_csv('C:/Columbia_University/Research/History_African_American_Migration_Pattern/US_MLP_datasets/1880 from 1880-1900 experiment.csv')
ds2 = pd.read_csv('C:/Columbia_University/Research/History_African_American_Migration_Pattern/US_MLP_datasets/1900 from 1880-1900 experiment.csv')

In [2]:
ds1["matching_id"] = None
ds2["matching_id"] = None

In [8]:
def create_dataframe(dataset1, dataset2):

    df = {index1: [] for index1, row1 in dataset1.iterrows()}
    

    for index1, row1 in dataset1.iterrows():
        score_list = []
        
        
        histid1 = row1['histid']
        firstname1 = row1['namefrst']
        lastname1 = row1['namelast']
        birthplace1 = row1['bpl']

        for index2, row2 in dataset2.iterrows():

            histid2 = row2['histid']
            firstname2 = row2['namefrst']
            lastname2 = row2['namelast']
            birthplace2 = row2['bpl']  

            histid_score =  rltk.levenshtein_similarity(histid1, histid2)*0.4
            firstname_score = rltk.levenshtein_similarity(firstname1, firstname2)*0.2
            lastname_score = rltk.levenshtein_similarity(lastname1, lastname2)*0.2

            if birthplace1 == birthplace2:
                birthplace_score = 1*0.2
            else:
                birthplace_score = 0*0.2

            total_score = histid_score + firstname_score + lastname_score + birthplace_score
            
            score_list.append(total_score)

        df[index1].extend(score_list)

    return pd.DataFrame(df)


#### Using the rltk algorithm to help us create a matrix showing a linkage score between any pair of records in two datasets. Higher the score, the more likely the two records become a match. 

In [15]:
score_df = create_dataframe(ds1, ds2)
create_dataframe(ds1, ds2)


,0,1,2,3,4
0,0.117460,0.168889,0.095238,0.055556,0.125000
1,0.077778,0.066667,0.197778,0.077778,0.111111
2,0.184444,0.120000,0.188889,0.077778,0.158333


#### We will include Gale Shapley algorithm below